In [1]:
!pip install mysql-connector-python fastapi nest-asyncio uvicorn pyngrok requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 41.5 MB/s eta 0:00:00


In [2]:
!pip install pandas openpyxl mysql-connector-python


In [3]:
pip install ctransformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 57.3 MB/s eta 0:00:00


In [22]:
import mysql.connector

conn = mysql.connector.connect(
    host="0.tcp.in.ngrok.io",
    port=16582,
    user="root",
    password="diya@321",
    database="office"
)

cursor = conn.cursor()
print(" Connected to MariaDB!")


 Connected to MariaDB!


In [6]:
cursor.execute("SELECT * FROM shipment_view LIMIT 10")
rows = cursor.fetchall()

for row in rows:
    print(row)


(25, '15N', '150N-I', 'Lubes', 'MR', 2024, 'Q2', 'Apr', Decimal('8.70'), Decimal('14.70'))
(25, '15N', '150N-I', 'Lubes', 'MR', 2024, 'Q2', 'Apr', Decimal('8.70'), Decimal('14.70'))
(25, '15N', '150N-I', 'Lubes', 'MR', 2024, 'Q2', 'Jun', Decimal('9.60'), Decimal('9.70'))
(25, '15N', '150N-I', 'Lubes', 'MR', 2024, 'Q2', 'Jun', Decimal('9.60'), Decimal('9.70'))
(25, '15N', '150N-I', 'Lubes', 'MR', 2024, 'Q2', 'May', Decimal('10.30'), Decimal('9.70'))
(25, '15N', '150N-I', 'Lubes', 'MR', 2024, 'Q2', 'May', Decimal('10.30'), Decimal('9.70'))
(25, '15N', '150N-I', 'Lubes', 'MR', 2024, 'Q3', 'Aug', Decimal('8.50'), Decimal('5.80'))
(25, '15N', '150N-I', 'Lubes', 'MR', 2024, 'Q3', 'Aug', Decimal('8.50'), Decimal('5.80'))
(25, '15N', '150N-I', 'Lubes', 'MR', 2024, 'Q3', 'Jul', Decimal('11.20'), Decimal('10.70'))
(25, '15N', '150N-I', 'Lubes', 'MR', 2024, 'Q3', 'Jul', Decimal('11.20'), Decimal('10.70'))


In [7]:
!pip install ctransformers
!apt-get install -y libopenblas-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libopenblas-dev is already the newest version (0.3.20+ds-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [8]:
from huggingface_hub import snapshot_download

model_path = snapshot_download(
    repo_id="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    allow_patterns="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    local_dir="/content/mistral-gguf"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

mistral-7b-instruct-v0.1.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

In [9]:
import re
from ctransformers import AutoModelForCausalLM

view_schema = """CREATE VIEW shipment_summary (
    product_id INT,
    product_code VARCHAR(10),
    product_name VARCHAR(50),
    product_type VARCHAR(50),
    product_site VARCHAR(50),
    YEAR INT,
    qtr VARCHAR(5),
    month VARCHAR(15),
    plan DECIMAL(10,2),
    actual DECIMAL(10,2)
);"""

def make_prompt(schema, question):
    return f"""### Task
You are an expert in SQL. Generate a valid MariaDB SQL query for the question using the given schema.

### View Schema
{schema}

### Question
{question}

### SQL
"""


In [10]:
from ctransformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "/content/mistral-gguf",
    model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    model_type="mistral"
)

In [11]:
import re

def generate_sql(question, schema):
    prompt = make_prompt(schema, question)
    response = model(prompt)
    match = re.search(r"``````", response, re.DOTALL | re.IGNORECASE)
    if match:
        sql = match.group(1).strip()
    else:
        sql = response.strip().split("### SQL")[-1].strip()
    return sql

In [12]:
view_schema = """CREATE VIEW shipment_summary (
    product_id INT,
    product_code VARCHAR(10),
    product_name VARCHAR(50),
    product_type VARCHAR(50),
    product_site VARCHAR(50),
    YEAR INT,
    qtr VARCHAR(5),
    month VARCHAR(15),
    plan DECIMAL(10,2),
    actual DECIMAL(10,2)
);"""

question = "Write a SQL query to get the total actual shipments per product_type for the year 2024 from the shipment_view table."

sql_query = generate_sql(question, view_schema)
print("Generated SQL:\n", sql_query)


Generated SQL:
 SELECT product_type, SUM(actual) as TotalActualShipments
FROM shipment_summary
WHERE YEAR = 2024
GROUP BY product_type;


In [13]:
create_table_query = """
CREATE TABLE IF NOT EXISTS chat_history (
    session_id VARCHAR(255),
    user_message TEXT,
    bot_response TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (session_id, timestamp)
) ENGINE=InnoDB;
"""
cursor.execute(create_table_query)
conn.commit()
print("Chat history table ready")

def save_chat(session_id, user_message, bot_response):
    insert_query = """
        INSERT INTO chat_history (session_id, user_message, bot_response)
        VALUES (%s, %s, %s)
    """
    cursor.execute(insert_query, (session_id, user_message, bot_response))
    conn.commit()

def load_chat_history(session_id, max_turns=5):
    select_query = """
        SELECT user_message, bot_response FROM chat_history
        WHERE session_id=%s
        ORDER BY timestamp DESC
        LIMIT %s
    """
    cursor.execute(select_query, (session_id, max_turns))
    rows = cursor.fetchall()[::-1]
    history = ""
    for user_msg, bot_resp in rows:
        history += f"User: {user_msg}\nBot: {bot_resp}\n"
    return history


Chat history table ready


In [14]:
def chatbot_response(session_id, user_message, schema):

    history = load_chat_history(session_id)

    prompt = (
        f"{history}"
        f"You are an expert in SQL. Use the schema below to answer the user's latest question.\n"
        f"Schema:\n{schema}\n"
        f"User: {user_message}\n"
        f"Bot:"
    )
    response = model(prompt)

    save_chat(session_id, user_message, response)
    return response


In [15]:
def chatbot_response(session_id, user_message, schema):
    # Load the last few turns of chat history
    history = load_chat_history(session_id)
    # Create a context-aware prompt for the LLM
    prompt = (
        f"{history}"
        f"You are an expert in SQL. Use the schema below to answer the user's latest question.\n"
        f"Schema:\n{schema}\n"
        f"User: {user_message}\n"
        f"Bot:"
    )
    # Get the model's response
    response = model(prompt)
    # Save the interaction to your chat history table
    save_chat(session_id, user_message, response)
    return response


In [16]:
!pip install fastapi uvicorn nest-asyncio

In [17]:
from pyngrok import ngrok

ngrok.set_auth_token("2zXbc9tZzMqXZaMbVwhDONxqJC6_5GpcxAiFbZhw9FYt5yts4")


In [18]:
!pip install fastapi nest-asyncio uvicorn pyngrok sse-starlette


In [42]:
!pkill ngrok

In [43]:
# ===================== SERVER CODE =====================
import nest_asyncio
import uvicorn
from fastapi import FastAPI, Request
from fastapi.responses import StreamingResponse
from ctransformers import AutoModelForCausalLM
from pyngrok import ngrok
import mysql.connector
from datetime import datetime

# Enable asyncio in Colab or Jupyter
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI()

# Set ngrok token and expose port 8090
ngrok.set_auth_token("2zXbc9tZzMqXZaMbVwhDONxqJC6_5GpcxAiFbZhw9FYt5yts4")
public_url = ngrok.connect(8090)
print("FastAPI Server Public URL:", public_url)

# Connect to MariaDB
conn = mysql.connector.connect(
    host="0.tcp.in.ngrok.io",
    port=14438,
    user="root",
    password="diya@321",
    database="office"
)
cursor = conn.cursor()

print("Loading model")
model = AutoModelForCausalLM.from_pretrained(
    "/content/mistral-gguf",
    model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    model_type="mistral"
)
print("Model loaded.")

# SQL Schema string
schema = """CREATE VIEW shipment_summary (
    product_id INT,
    product_code VARCHAR(10),
    product_name VARCHAR(50),
    product_type VARCHAR(50),
    product_site VARCHAR(50),
    YEAR INT,
    qtr VARCHAR(5),
    month VARCHAR(15),
    plan DECIMAL(10,2),
    actual DECIMAL(10,2)
);"""

# Ensure chat_history table exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS chat_history (
    session_id VARCHAR(255),
    user_message TEXT,
    bot_response TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (session_id, timestamp)
) ENGINE=InnoDB;
""")
conn.commit()

# Save chat in DB
def save_chat(session_id, user_message, bot_response):
    insert_query = """
        INSERT INTO chat_history (session_id, user_message, bot_response)
        VALUES (%s, %s, %s)
    """
    cursor.execute(insert_query, (session_id, user_message, bot_response))
    conn.commit()

# Load last few chat turns
def load_chat_history(session_id, max_turns=5):
    select_query = """
        SELECT user_message, bot_response FROM chat_history
        WHERE session_id=%s
        ORDER BY timestamp DESC
        LIMIT %s
    """
    cursor.execute(select_query, (session_id, max_turns))
    rows = cursor.fetchall()[::-1]
    history = ""
    for user_msg, bot_resp in rows:
        history += f"User: {user_msg}\nBot: {bot_resp}\n"
    return history

# Streaming response generator
def stream_response(session_id, user_message):
    history = load_chat_history(session_id)
    prompt = (
        f"{history}"
        f"You are an expert in SQL. Use the schema below to answer the user's latest question.\n"
        f"Schema:\n{schema}\n"
        f"User: {user_message}\n"
        f"Bot:"
    )
    print(f"\n PROMPT SENT TO MODEL:\n{prompt}\n")

    response_text = ""
    for chunk in model(prompt, stream=True):
        print(chunk, end="", flush=True)
        yield chunk
        response_text += chunk
    save_chat(session_id, user_message, response_text)

# FastAPI /chat route
@app.post("/chat")
async def chat(request: Request):
    data = await request.json()
    session_id = data.get("session_id")
    user_message = data.get("message")

    print(f"\n Received from client | session: {session_id} | message: {user_message}")
    return StreamingResponse(
        stream_response(session_id, user_message),
        media_type="text/plain"
    )

# Start FastAPI server
uvicorn.run(app, port=8090)


FastAPI Server Public URL: NgrokTunnel: "https://484177e50d30.ngrok-free.app" -> "http://localhost:8090"
Loading model
Model loaded.


INFO:     Started server process [160]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8090 (Press CTRL+C to quit)



 Received from client | session: user_session_001 | message: hi
INFO:     34.75.174.168:0 - "POST /chat HTTP/1.1" 200 OK

 PROMPT SENT TO MODEL:
User: hi
Bot:  Hello! How can I assist you today?
User: give sql query for giving the count of all rows in the table shipment_view
Bot:   Here's an SQL query to retrieve the count of all rows in the `shipment_view` table:
```sql
SELECT COUNT(*) FROM shipment_view;
```
This query will return a single row with a single column, containing the total number of rows in the table.
User: sql query to print number of rows
Bot:  Here's an SQL query to get the count of rows in the `shipment_summary` table:
```sql
SELECT COUNT(*) FROM shipment_summary;
```
This query will return a single row with a single column, containing the total number of rows in the table.
You are an expert in SQL. Use the schema below to answer the user's latest question.
Schema:
CREATE VIEW shipment_summary (
    product_id INT,
    product_code VARCHAR(10),
    product_name VARC

 Use

 the

...

 Use

 the

 schema

 creation

ERROR:    Exception in ASGI application
  + Exception Group Traceback (most recent call last):
  |   File "/usr/local/lib/python3.11/dist-packages/starlette/_utils.py", line 77, in collapse_excgroups
  |     yield
  |   File "/usr/local/lib/python3.11/dist-packages/starlette/responses.py", line 271, in __call__
  |     async with anyio.create_task_group() as task_group:
  |   File "/usr/local/lib/python3.11/dist-packages/anyio/_backends/_asyncio.py", line 772, in __aexit__
  |     raise BaseExceptionGroup(
  | ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "/usr/local/lib/python3.11/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    |     result = await app(  # type: ignore[func-returns-value]
    |              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    |   File "/usr/local/lib/python3.11/dist-packages/uvicorn/middleware/proxy_headers.py",